In [28]:
import numpy as np
import pandas as pd
import xlwings as xw

In [29]:
hebing = [
    '中国航天科工集团',
'中国电子科技集团',
'中国航天科技集团',
'中国船舶集团',
'中国航空工业集团',
'中国电子信息产业集团',
'中国兵器工业集团',
'杨茂富',
'智明达',
'上海瀚讯',
'江滔',
'唐富奎',
'藏朝峰',
'王晓燕',
'杜春香',
'香港艾特技術有限公司',
'王柯',
'Orbis',
'Super Chip',
'刘英',
'李海鹏',
'楼继勇',
'康达新材',
'中国兵器装备集团',
'陈建新'
]

In [30]:
df = pd.read_excel('../收入成本大表2020-2022.xlsx', sheet_name='2022')
df.dtypes

序号                   int64
日期          datetime64[ns]
销售主体                object
年度                   int64
月份                   int64
发票号码                object
客户代码                object
客户名称                object
工商名称                object
合并口径                object
客户名称（原始）            object
销售类型                object
客户类型1               object
客户类型2               object
合并口径.1              object
客户代码.1              object
地区                  object
主营/其他               object
产品代码                object
从杰瑞采购              float64
产品名称                object
砖型                  object
档位                  object
数量                   int64
销售含税单价             float64
不含税销售额             float64
审定总成本              float64
规格型号               float64
批号                  object
销售订单号               object
源单单号                object
合同号                 object
合同签订时间             float64
出库日期        datetime64[ns]
发货仓库                object
凭证号                 object
业务员                 object
F

In [31]:
comp = {}  # comp[合并口径]=下属公司 1、客户年总销售量大于20万
for i in hebing:
    temp = df[df['合并口径']==i].groupby('工商名称')['不含税销售额'].agg(np.sum).sort_values()
    comp_name = temp[temp>=200000].index
    comp[i] =comp_name

In [32]:
n = 0
for i in range(len(hebing)):
    a = len(comp[hebing[i]])
    n += a    
n

72

In [33]:
contracts ={}  # contracts[公司名称]=合同编号 2、抽取金额最大的三个合同，
for i in hebing:
    for j in comp[i]:
        temp = df[(df['工商名称']==j) & (df['合并口径']==i)].groupby('合同号')['不含税销售额'].agg(np.sum).sort_values(ascending=False)
        if len(temp)<=3:
            contracts[j] = temp.index
        else:
            contracts[j] = temp.index[0:3]
            

In [34]:
out = pd.DataFrame()
for i in hebing: # 集团
    for j in comp[i]: # 公司
        for c_id in contracts[j]: # 合同
            temp = df[(df['合同号']==c_id) & (df['合并口径']==i) & (df['工商名称']==j)]           
            out = pd.concat([out, temp])
out.shape

(1088, 39)

In [35]:
out['合同金额'] = out['数量']*out['销售含税单价']
out.rename(columns={'源单单号':'出库单号', 'FNote': '快递单号'}, inplace=True)

In [36]:
out = out[['合并口径','工商名称','合同号','产品代码','数量','销售含税单价','合同金额',\
    '出库日期', '出库单号','不含税销售额' ,'日期' ,'发票号码','凭证号','快递单号']]

In [37]:
out.dtypes

合并口径              object
工商名称              object
合同号               object
产品代码              object
数量                 int64
销售含税单价           float64
合同金额             float64
出库日期      datetime64[ns]
出库单号              object
不含税销售额           float64
日期        datetime64[ns]
发票号码              object
凭证号               object
快递单号              object
dtype: object

In [38]:
# out.to_excel('../out.xlsx', index=False, encoding='utf-8')
out.shape

(1088, 14)

In [39]:
dongfang = pd.read_excel('../东方-收入测试-2022.xlsx', skiprows=3, usecols=[i for i in range(24)])
dongfang.fillna('', inplace=True)
dongfang.dtypes

客户名称                object
客户代码/名称             object
底稿索引号               object
收入凭证年月              object
凭证号                 object
收入金额               float64
合同号                 object
标的物型号               object
单个售价               float64
合同数量                 int64
合同金额（含税）           float64
出库时间                object
出库单号                object
发票号码                object
开票日期        datetime64[ns]
客户签收确认时间            object
快递公司                object
快递单号                object
发件时间                object
回款凭证日期              object
回款凭证号               object
对方科目                object
回款金额                object
票据类型                object
dtype: object

In [40]:
dongfang.rename(columns={'客户名称':'工商名称','标的物型号':'产品代码','合同数量':'数量','合同金额（含税）':'合同金额'}, inplace=True)

In [41]:
new = out.merge(dongfang, how='left', on=['工商名称','合同号','产品代码','数量','出库单号'])
new.shape

(1088, 33)

In [42]:
new.columns

Index(['合并口径', '工商名称', '合同号', '产品代码', '数量', '销售含税单价', '合同金额_x', '出库日期', '出库单号',
       '不含税销售额', '日期', '发票号码_x', '凭证号_x', '快递单号_x', '客户代码/名称', '底稿索引号',
       '收入凭证年月', '凭证号_y', '收入金额', '单个售价', '合同金额_y', '出库时间', '发票号码_y', '开票日期',
       '客户签收确认时间', '快递公司', '快递单号_y', '发件时间', '回款凭证日期', '回款凭证号', '对方科目', '回款金额',
       '票据类型'],
      dtype='object')

In [43]:
new['销售收入凭证'] = new['收入凭证年月']+'/'+new['凭证号_y']

In [44]:
new['货款结算方式'] = new['回款凭证号']+'/'+new['对方科目']+'/'+new['票据类型']

In [46]:
new = new[['合并口径','工商名称','合同号','产品代码','数量','销售含税单价','合同金额_x','出库日期','出库单号',\
    '快递公司', '快递单号_y','发件时间','客户签收确认时间','开票日期','发票号码_x','不含税销售额',\
        '销售收入凭证', '收入金额','货款结算方式','底稿索引号','回款凭证日期','回款金额']]

In [47]:
new.to_excel('../new.xlsx', index=0, encoding='utf-8')

In [ ]:
# 图片转pdf之后合并的代码